In [1]:
from pyseqrna import pyseqrna_utils as pu
from pyseqrna import quality_check as qc
from pyseqrna import quality_trimming as qt
from pyseqrna import  aligners as al
from pyseqrna import pyseqrna_stats as ps
from pyseqrna import quants
from pyseqrna import differential_expression as de
from pyseqrna import pyseqrna_plots as pp
import pandas as pd
import dill

In [ ]:
from pyseqrna.pyseqrna_utils import PyseqrnaLogger

log = PyseqrnaLogger(mode='w', log='pp')

log.info("Analysis started")

In [ ]:
data = pu.read_input_file("pyseqrna/example/input_Sample_PE.txt", "pyseqrna/example/data/" , paired=True)
samples= data['samples']

In [ ]:
dd= pd.read_excel("pySeqRNA_results/filtered_DEGs.xlsx")

In [ ]:
targets= data['targets']
comb= data['combinations']

In [ ]:
dill.dump_session("test.pyseqrna")

In [2]:
dill.load_session("test.pyseqrna")

In [ ]:
pp.plotVenn(DEGFile="pySeqRNA_results/filtered_DEGs.xlsx", comparisons=['C1-D6','D1-C6','H1-C6','D1-H6' ], FOLD=1,  outDir="pySeqRNA_results")

In [ ]:
pu.make_directory("pySeqRNA_results")

In [ ]:

a = qt.trim_galoreRun(sampleDict=samples, paired=True, slurm=True)

In [ ]:
aligner = al.STAR_Aligner(genome="pyseqrna/example/data/arabidopsis10.fasta", slurm=True)

In [ ]:
j = aligner.build_index()

In [ ]:
aligner.check_index()

In [ ]:
outalign, job = aligner.run_Alignment(target=a[0], pairedEND=True)

In [ ]:
df= ps.align_stats(sampleDict=samples,trimDict=a[0],bamDict=outalign, pairedEND=True)

In [ ]:
df.to_excel("pySeqRNA_results/alignment_stats.xlsx", index=False)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import seaborn as sns

In [ ]:
sample = list(df['Sample'])
creads = list(df['Input_reads2x'])
iread = list(df['%_Cleaned2x'])
tc= list(100-df['%_Cleaned2x'])

plt.barh(sample,iread,0.6,label="cleaned Reads", color='skyblue')
plt.barh(sample, tc, 0.6,left=iread, label="discarded", color='tomato')
plt.margins(y=0.01)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("% Number of reads", fontsize=18)
plt.ylabel("Samples", fontsize=18)
plt.legend(['Cleaned', 'Discarded'], bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=15)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)

# plt.legend(frameon=False, fontsize=17)



In [ ]:
tc

In [ ]:
a= quants.featureCount(bamDict=outalign, gff="pyseqrna/example/data/arabidopsis10.gff", outDir="pySeqRNA_results", slurm=True)

In [ ]:
counts= pd.read_csv("pySeqRNA_results/Counts_final.txt", sep="\t")

In [ ]:
counts

In [ ]:
targets= data['targets']
comb= data['combinations']

In [ ]:
dd= de.runDESeq2(countDF=counts,targetFile=targets,design='sample', combination=comb)

In [ ]:
import os 
dd.to_excel(os.path.join("pySeqRNA_results","Raw_DEGs_all.xlsx"), index=False)

In [ ]:
dd= pd.read_excel("pySeqRNA_results/Raw_DEGs_all.xlsx")
filtered_DEG = de.degFilter(degDF=dd, CompareList=comb, FDR=0.05, FOLD=1)

In [ ]:
summary= filtered_DEG['summary']

In [ ]:
wd= pd.ExcelWriter(os.path.join("pySeqRNA_results","filtered_DEGs.xlsx"))
for key, value in filtered_DEG['filtered'].items():
    value.to_excel(wd,sheet_name=key)
    wd.save()
wd.close()

In [ ]:
pu.getGenes(os.path.join("pySeqRNA_results","filtered_DEGs.xlsx"), combinations=comb)

In [ ]:
from pyseqrna import normalize_counts as nc

In [ ]:
rpkm = nc.Normalization(countFile="pySeqRNA_results/Counts_final.txt", featureFile="pyseqrna/example/data/arabidopsis10.gff")

In [ ]:
rpk = rpkm.meanRatioCount()

In [ ]:
rpk.to_excel("pySeqRNA_results/rpkm.xlsx")

In [ ]:
rcount= pd.read_excel("pySeqRNA_results/rpkm.xlsx")

In [ ]:
rcount

In [ ]:
from pyseqrna import pyseqrna_plots as pp

In [ ]:
heatmap, ax = pp.plotHeatmap(dd,combinations=comb,num=50, type='deg')

In [ ]:
from pyseqrna import gene_ontology as go

In [ ]:
bdata= go.query('athaliana')

In [ ]:
gdata = go.preprocessBioMart(bdata)

In [ ]:
results = go.enrichGO('athaliana',"pySeqRNA_results/diff_genes/C1-D6.txt")

In [ ]:
results

In [ ]:
df=bdata
import numpy as np
df2 = df[df['GO_ID'].notna()]
gg = list(df2['Gene'])
x = np.array(gg)

bg_count = len(np.unique(x))

bg_count


In [ ]:
cd= pd.read_excel("pySeqRNA_results.1/M1-V6_deg.xlsx")

In [ ]:
gdata

In [ ]:
from pyseqrna import pathway


In [ ]:
dk = pathway.enrichKEGG(file="pySeqRNA_results/diff_genes/C1-D6.txt", species='ath')